# Predicting United States Real Estate Prices
By: Grace Li, Olivia Weisiger and Fionnuala Eastwood

### Outline
1. Dataset Preprocessing and Merging
2. Data Visualization
3. Classic Machine Learning Models
4. Deep Learning Models
5. Analysis of accuracy and results

### Context
The United States housing market is...

### Our Goal

This project will explore the current United States real-estate market, investigate what factors influence the price of property, and create multiple machine learning models that predict these housing costs throughout the country. More specifically, this will be accomplished through implementation of (add briefly about what models we end up using....) Being able to infer and understand the trends of real estate is extremely valuable economic knowledge that will provide important insights about our country. 

Furthermore, our project aims to deepen our understanding of how societal biases influence external structures such as the economy. By merging datasets, we will investigate which underlying factors such as (add briefly when choose other data) affect the prices of houses in order to draw deeper conclusions about intangible factors impacting our economic climate.

### Import Packages and Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

This data is from Kaggle's "USA Real Estate Dataset" found here: https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset

In [2]:
df = pd.read_csv('realtor-data.zip.csv')

### Initial Data Processing
Let's first break down what our dataset looks like...

In [3]:
df.shape

(2226382, 12)

We have a dataset with over 2 million rows and 12 columns. Since this is way too many samples to process in a reasonable computational time, we will take a random subset of 100,000 of these samples to perform analysis on.

In [4]:
df = df.sample(100000)

With our refined sample, let's get an idea of what our dataset looks like by outputting a few rows of the table.

In [5]:
df.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
538226,23756.0,for_sale,12000.0,NaN,NaN,0.24,1132674.0,Lake Placid,Florida,33852.0,NaN,2004-08-02
1091797,27192.0,for_sale,25000.0,NaN,NaN,0.75,1922843.0,Madisonville,Texas,77864.0,NaN,NaN
475856,63439.0,for_sale,33400.0,NaN,NaN,NaN,913015.0,Palm Bay,Florida,32908.0,NaN,2021-02-17
463034,30750.0,for_sale,365000.0,3.0,3.0,0.05,1243568.0,Sanford,Florida,32771.0,1944.0,2017-11-03
1238568,22415.0,for_sale,150000.0,NaN,NaN,8.31,1033413.0,Tijeras,New Mexico,87059.0,NaN,NaN


Notice that each sample in the dataset is a real estate listing in the United States (the listings are all from 2022-2024), and each sample has 12 features that provide numerical or categorical information about the listing.

Here is an overview of each feature's meaning and data type:

- brokered_by:

- status:

- price:

- bed:

- bath:

- etc fill in later (look on kaggle these descriptins are provided)

Now that we have an understanding of our data set, we will perform some processing on the data so that it is cleaner to use. Firstly, we will drop some unnecessary columns that do not contribute to our analysis goals. The brokered_by column which encodes the real-estate company in charge of the property is not necessary because we are interested in the qualities of the house itself. Additionally, the status column is not needed because we will use the price set for the house equivalently regardless if it is sold or for sale. Lastly, the previously sold date can be dropped since we are focused on the current selling price. We will trim our dataset from 12 columns to 9 with these modifications.

In [6]:
df = df.drop(['brokered_by', 'status', 'prev_sold_date'], axis=1)

This dataset contains listings from the United States and all it's territories. For our purposes, we only want to analyze data from the 50 states (and Washington, DC) so let's trim out samples taken from Puerto Rico and the Virgin Islands.

In [7]:
df = df[(df['state'] != "Puerto Rico") & (df['state'] != "Virgin Islands")]

Our next processing step is making sure we don't have any NaN's in our dataset, as empty data values might impact our analysis models.

In [8]:
#sum up all NaN values present in dataset (in any feature column)
print (df.isnull().sum().sum())

85381


We see that we have some data entries with no value, so let's remove all rows that contain any NaN values. We will also check the shape of our data frame after this removal to make sure we still have plenty of samples to work with.

In [9]:
#remove all rows missing data
df = df.dropna()

#verify we now have no NaN values, expect a value of zero
print (f'We now have: {df.isnull().sum().sum()} NaN entires')

#print new shape
print (f'Our new dataset shape is {df.shape}')


We now have: 0 NaN entires
Our new dataset shape is (60732, 9)


We successfully dropped all empty entries and still have a substantial size data frame to analyze.

Our last step in data processing is preparing our target price data for our future machine learning models. We noticed that predicting the price to an exact number (as the current column does) is quite specific, so instead in some cases we will want to predict whether any piece of real-estate is more generally expensive or cheap. The next question that follows is how we will quantify this "expensive" vs "cheap". 

Our natural thought was just categorizing the samples based on if they were on the higher half of all in our dataset vs the lowest. However, upon further analysis we realize that the state the property is in has an overwhelmingly powerful influence on this categorization. For example we would see that practically all samples from New York would fall in the upper portion of data, while a huge majority of samples from rural states will be in the lower. This would leave our model with very little to do, so to work around this we have decided to create a categorical column that contains a 1 if the property price is above the median housing price **of the state it is in**, and a 0 if the property is below this median average of its state. This takes out the state bias and may lead to more informative conclusions about other features that are no longer overshadowed.

To do this, we first merge data with the median housing prices by state. This data was taken from the following link: https://www.bankrate.com/real-estate/median-home-price/#how-much, and is up to date as of November 2023 (which is the same time period our real-estate data was taken from).

In [10]:
#upload data
df_med = pd.read_csv('median_prices.csv')
df_med.head()

#select state and median price columns we want
df_med = df_med[["state", "med_price"]]

#merge along state column
df = pd.merge(df, df_med, on = ["state"])
df.head()

,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,med_price
0,365000.0,3.0,3.0,0.05,1243568.0,Sanford,Florida,32771.0,1944.0,"$405,000"
1,375000.0,4.0,2.0,0.14,1367143.0,Lake Worth,Florida,33467.0,1512.0,"$405,000"
2,6350000.0,6.0,7.0,0.19,475321.0,Miami,Florida,33133.0,4600.0,"$405,000"
3,259900.0,3.0,2.0,0.26,214892.0,Deland,Florida,32724.0,1205.0,"$405,000"
4,269000.0,3.0,2.0,0.21,1655670.0,Orlando,Florida,32809.0,1920.0,"$405,000"


Notice one more problem exists: we have a price listed in string form with dollar sign and commas. Instead we want it to be numerical in order to compare it with our current price column.

In [11]:
# Remove dollar signs and commas, then convert to integers
df['med_price'] = df['med_price'].replace({'\$': '', ',': ''}, regex=True).astype(int)

Lastly, we want to create a new column which we will call above_average. This column will contain a 1 if the price of that sample is above the median price in the state, and a 0 if it is below.

In [12]:
df['above_average'] = df.apply(lambda row: 1 if row['price'] > row['med_price'] else 0, axis=1)
df.head()

,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,med_price,above_average
0,365000.0,3.0,3.0,0.05,1243568.0,Sanford,Florida,32771.0,1944.0,405000,0
1,375000.0,4.0,2.0,0.14,1367143.0,Lake Worth,Florida,33467.0,1512.0,405000,0
2,6350000.0,6.0,7.0,0.19,475321.0,Miami,Florida,33133.0,4600.0,405000,1
3,259900.0,3.0,2.0,0.26,214892.0,Deland,Florida,32724.0,1205.0,405000,0
4,269000.0,3.0,2.0,0.21,1655670.0,Orlando,Florida,32809.0,1920.0,405000,0


This looks good, now we are ready to merge with other data sets to add more features to analyze.

### Dataset Merging

While the relationship between features such as number of rooms or number of acres on real-estate prices is quite intuitive, this project aims to delve beyond these variables and investigate more abstract influences. This will be done by merging our current dataframe with new datasets in order to add features including minimum wage of the state, median income by zip code, and even political affiliation, as we are curious if any of these variables will display a strong correlation with housing prices. One caution to note is that our original real estate data is from the past two years, so we will need to make sure the data we are merging with is taken from the same time period in order to obtain accurate conclusions.

The first dataset we will merge with is Kaggle's "US Household Income by Zip Code 2021-2011" found here: https://www.kaggle.com/datasets/claygendron/us-household-income-by-zip-code-2021-2011

In [13]:
df_income = pd.read_csv('us_income_zipcode.csv')
df_income.head()

,ZIP,Geography,Geographic Area Name,Households,Households Margin of Error,"Households Less Than $10,000","Households Less Than $10,000 Margin of Error","Households $10,000 to $14,999","Households $10,000 to $14,999 Margin of Error","Households $15,000 to $24,999",...,"Nonfamily Households $150,000 to $199,999","Nonfamily Households $150,000 to $199,999 Margin of Error","Nonfamily Households $200,000 or More","Nonfamily Households $200,000 or More Margin of Error",Nonfamily Households Median Income (Dollars),Nonfamily Households Median Income (Dollars) Margin of Error,Nonfamily Households Mean Income (Dollars),Nonfamily Households Mean Income (Dollars) Margin of Error,Nonfamily Households Nonfamily Income in the Past 12 Months,Year
0,601,860Z200US00601,ZCTA5 00601,5397.0,264.0,33.2,4.4,15.7,2.9,23.9,...,0.0,2.8,0.0,2.8,9386.0,1472.0,13044.0,1949.0,15.0,2021.0
1,602,860Z200US00602,ZCTA5 00602,12858.0,448.0,27.1,2.9,12.7,2.1,20.5,...,0.0,1.3,0.0,1.3,11242.0,1993.0,16419.0,2310.0,20.1,2021.0
2,603,860Z200US00603,ZCTA5 00603,19295.0,555.0,32.1,2.5,13.4,1.6,17.2,...,0.6,0.6,0.2,0.4,10639.0,844.0,16824.0,2217.0,34.9,2021.0
3,606,860Z200US00606,ZCTA5 00606,1968.0,171.0,28.4,5.5,13.3,4.4,23.3,...,0.0,7.5,0.0,7.5,15849.0,3067.0,16312.0,2662.0,13.0,2021.0
4,610,860Z200US00610,ZCTA5 00610,8934.0,372.0,20.5,2.5,13.2,2.5,23.3,...,0.0,1.8,0.0,1.8,12832.0,2405.0,16756.0,1740.0,14.5,2021.0


This dataset contains the results of the 2011 and 2021 national census, and we have chosen it in order to add a median income feature to our real estate pricing dataset. As explained above, we are only interested in the 2021 data since our pricing data comes from recent years, so we will trim down our dataset accordingly. Additionally, the dataset comes with dozens of feature columns, but for our purposes we only need to keep the zip code column (which we will use to merge our original dataset), and the median household income column. So let's process our dataset and display the cleaner result.

In [14]:
#select only samples from most recent census
df_income = df_income[df_income["Year"] == 2021.0]

#select only features we want
df_income = df_income[["ZIP", "Nonfamily Households Median Income (Dollars)"]]

df_income.head()

,ZIP,Nonfamily Households Median Income (Dollars)
0,601,9386.0
1,602,11242.0
2,603,10639.0
3,606,15849.0
4,610,12832.0


Now we are ready to merge with our original dataset. Currently our zip code columns have different names so we will rename them identically, and they also have different types (integer vs float) so we will convert to a float variable to avoid type error interference.

In [15]:
df_income["ZIP"] = df_income["ZIP"].astype(float)

df_income = df_income.rename(columns={'ZIP': 'zip_code'})

We will use an inner merge (explain why...)
The census data was very thorough (we have very few NaN values), so we can just remove any empty data rows and our dataset remains practically the same. We verify this assumption by outputting our dataset shape after the merge.

In [16]:
df = pd.merge(df, df_income, on = ["zip_code"])

#remove all rows missing data
df = df.dropna()

print (df.shape)
df.head()

(59983, 12)


,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,med_price,above_average,Nonfamily Households Median Income (Dollars)
0,365000.0,3.0,3.0,0.05,1243568.0,Sanford,Florida,32771.0,1944.0,405000,0,39346.0
1,259900.0,4.0,3.0,0.12,681517.0,Sanford,Florida,32771.0,1615.0,405000,0,39346.0
2,250000.0,3.0,3.0,0.07,380205.0,Sanford,Florida,32771.0,1914.0,405000,0,39346.0
3,569000.0,4.0,4.0,0.14,1328277.0,Sanford,Florida,32771.0,1960.0,405000,1,39346.0
4,397900.0,3.0,2.0,0.11,1239170.0,Sanford,Florida,32771.0,1655.0,405000,0,39346.0


This feature looks good, let's move on to some more merges.

Next, we want to add to our dataset statistics on political affiliation by state and minimum wage by state, which should be slightly simpler than merging by zipcode. 

First we will use is Kaggle's "2020 US Presidential Election Results by State" linked here: https://www.kaggle.com/datasets/callummacpherson14/2020-us-presidential-election-results-by-state. This data was taken appropriately recently to match our real-estate data, and it contains voting percentage and win vs loss data on Biden and Trump from the 2020 election.


In [17]:
df_election = pd.read_csv('voting.csv.xls')
df_election.head()

,state,state_abr,trump_pct,biden_pct,trump_vote,biden_vote,trump_win,biden_win
0,Alaska,AK,53.1,43.0,189543,153502,1,0
1,Hawaii,HI,34.3,63.7,196864,366130,0,1
2,Washington,WA,39.0,58.4,1584651,2369612,0,1
3,Oregon,OR,40.7,56.9,958448,1340383,0,1
4,California,CA,34.3,63.5,5982194,11082293,0,1


Notice this is quite a clean dataset already, all we need to do is select the columns we are interested in and perform another inner merge along the column column of state. Here, we will choose to keep the state column which is needed for the merge, as well as the Trump pct, Biden pct columns since these provide more detailed information then the binary win vs loss columns. Let's do so and check our new dataset.

In [18]:
#select only features we want
df_election = df_election[["state", "biden_pct", "trump_pct"]]

#merge dataframe along the column of state
df = pd.merge(df, df_election, on = ["state"])

#verify there were no null data values added
print (f'We still have: {df.isnull().sum().sum()} NaN entires')

#output model summary
print (df.shape)
df.head()

We still have: 0 NaN entires
(59983, 14)


,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,med_price,above_average,Nonfamily Households Median Income (Dollars),biden_pct,trump_pct
0,365000.0,3.0,3.0,0.05,1243568.0,Sanford,Florida,32771.0,1944.0,405000,0,39346.0,47.9,51.2
1,259900.0,4.0,3.0,0.12,681517.0,Sanford,Florida,32771.0,1615.0,405000,0,39346.0,47.9,51.2
2,250000.0,3.0,3.0,0.07,380205.0,Sanford,Florida,32771.0,1914.0,405000,0,39346.0,47.9,51.2
3,569000.0,4.0,4.0,0.14,1328277.0,Sanford,Florida,32771.0,1960.0,405000,1,39346.0,47.9,51.2
4,397900.0,3.0,2.0,0.11,1239170.0,Sanford,Florida,32771.0,1655.0,405000,0,39346.0,47.9,51.2


Notice we still have no NaN entries, so our merging didn't add any problematic data and appears to be successfully added. 

Finally, we will perform this process one more time in order to add data on what each state's minimum wage is. This time we will use Kaggle's "Living Wage - State Capitals" found at https://www.kaggle.com/datasets/brandonconrady/living-wage-state-capitals. We again verified this was taken from the past two years for consistency.

In [19]:
df_minwage = pd.read_csv('LivingWageStateCapitals.csv.xls')
df_minwage.head()

,state_territory,city,minimum_wage,one_adult_no_kids_living_wage,one_adult_one_kid_living_wage,one_adult_two_kids_living_wage,one_adult_three_kids_living_wage,two_adults_one_working_no_kids_living_wage,two_adults_one_working_one_kid_living_wage,two_adults_one_working_two_kids_living_wage,...,one_adult_two_kids_poverty_wage,one_adult_three_kids_poverty_wage,two_adults_one_working_no_kids_poverty_wage,two_adults_one_working_one_kid_poverty_wage,two_adults_one_working_two_kids_poverty_wage,two_adults_one_working_three_kids_poverty_wage,two_adults_both_working_no_kids_poverty_wage,two_adults_both_working_one_kid_poverty_wage,two_adults_both_working_two_kids_poverty_wage,two_adults_both_working_three_kids_poverty_wage
0,District of Columbia,Washington,13.25,19.97,38.95,48.99,63.96,29.61,34.55,38.32,...,10.44,12.60,8.29,10.44,12.60,14.75,4.14,5.22,6.30,7.38
1,Alabama,Montgomery,7.25,13.56,27.35,33.42,42.17,22.59,26.66,30.27,...,10.44,12.60,8.29,10.44,12.60,14.75,4.14,5.22,6.30,7.38
2,Alaska,Juneau,10.19,15.48,29.99,36.00,47.42,24.48,29.46,33.01,...,13.05,15.75,10.36,13.05,15.75,18.44,5.18,6.53,7.87,9.22
3,Arizona,Phoenix,12.00,15.41,29.44,35.40,46.01,24.85,29.25,32.98,...,10.44,12.60,8.29,10.44,12.60,14.75,4.14,5.22,6.30,7.38
4,Arkansas,Little Rock,10.00,13.97,28.81,35.49,45.33,23.21,27.66,31.36,...,10.44,12.60,8.29,10.44,12.60,14.75,4.14,5.22,6.30,7.38


Again, we want to select the columns we need which in this case is the state column to merge along and the minimum_wage column which has the minimum wage data we desire (in dollars). Here, we will also rename the "state_territory" column to have the same title "state" as our original dataframe to streamline the merging process. Then after we complete the inner merge we will verify our final dataset.

In [20]:
#select only features we want
df_minwage = df_minwage[["state_territory", "minimum_wage"]]

#rename state_territory column
df_minwage = df_minwage.rename(columns={'state_territory': 'state'})

#merge dataframe along the column of state
df = pd.merge(df, df_minwage, on = ["state"])

#verify there were no null data values added
print (f'We still have: {df.isnull().sum().sum()} NaN entires')

#output model summary
print (df.shape)
df.head()

We still have: 0 NaN entires
(59983, 15)


,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,med_price,above_average,Nonfamily Households Median Income (Dollars),biden_pct,trump_pct,minimum_wage
0,365000.0,3.0,3.0,0.05,1243568.0,Sanford,Florida,32771.0,1944.0,405000,0,39346.0,47.9,51.2,8.56
1,259900.0,4.0,3.0,0.12,681517.0,Sanford,Florida,32771.0,1615.0,405000,0,39346.0,47.9,51.2,8.56
2,250000.0,3.0,3.0,0.07,380205.0,Sanford,Florida,32771.0,1914.0,405000,0,39346.0,47.9,51.2,8.56
3,569000.0,4.0,4.0,0.14,1328277.0,Sanford,Florida,32771.0,1960.0,405000,1,39346.0,47.9,51.2,8.56
4,397900.0,3.0,2.0,0.11,1239170.0,Sanford,Florida,32771.0,1655.0,405000,0,39346.0,47.9,51.2,8.56


Now we are officially done with merging our dataset and have plenty of new columns to work with!

### Data Visualization

(Just writing some notes for us to use later)
- maybe create a fancy visual heatmap type thing showing our prices by zipcode on the us map
- Create bar plots, histograms, correlation plots etc using tangible factors from our og dataset (room number, acres etc) should show clear trend
- Same thing but for some intangible factors using newly merged data, see if we can come to cool conclusions about those correlations
- Writeup analysis about what this shows us about society/housing market